
# Get coordinates for houses from mow-reestr


In [5]:
import json
import requests

In [9]:
import json
import requests

G_API_MAXIMUM = 0 #  бесплатный лимит
google_api = "" # личный ключ
google_url = "https://maps.googleapis.com/maps/api/geocode/json?address=4/"
google_url_end = "&key="

yandex_api = "" # личный ключ
Y_API_MAXIMUM = 999 #  бесплатный лимит
yandex_url = "https://geocode-maps.yandex.ru/1.x?format=json&geocode="
yandex_url_end="&apikey="

errors = "data/mow-reestr-geom/mow_reestr_geom_ERRORS.csv"


In [7]:
def get_dirs(num):
    return f'data/mow-reestr-geom/mow_reestr_geom_{num}.csv', f'data/mow-reestr-geom/mow_reestr_geom_{num}.csv'


## Parsing using google:

In [10]:
def get_coordinates_google(addr, num, line_num, requests_count):
    """
    :param addr: адрес
    :param num: номер файла для записи
    :param line_num: номер строки
    :param requests_count: счетчик запросов
    :returns: долгота, широта, счетчик запросов
    """
    try:
        if requests_count == G_API_MAXIMUM:
            print("limit exceeded ", num, " line_num: ", line_num)
            exit(0)
        r = requests.get(google_url + addr + google_url_end + google_api)
        resp = json.loads(r.text)

        lon = resp["results"][0]["geometry"]["location"]["lng"]
        lat = resp["results"][0]["geometry"]["location"]["lat"]
        requests_count += 1
        return lon, lat, requests_count
    except:
        print("error parsing ",num, "line num ", line_num, "addr ", addr)

In [13]:
duplicates = set()

def g_process_file(num, rc, sep=';'):
    """
    :param num: номер файла для записи
    :param rc: счетчик запросов
    :param sep: разделитель csv
    :returns: счетчик запросов
    """
    print("start ", num)
    input_file, output_file = get_dirs(num)
    
    out = open(output_file, 'a')
    errors = open(errors, 'a')
    with open(input_file) as f:
        line_num = 0
        for line in f:
            if line_num == 0:
                line_num += 1
                out_line = line.replace("\n", "") + sep + "lon" + sep + "lat" + "\n"
                out.write(out_line)
                continue

            split_line = line.split(sep)
            addr = split_line[0]
            if addr not in duplicates:
                form_addr = addr.replace("/", "%2F")
                form_addr = form_addr.replace(" ", "+")
                try:
                    lon, lat, rc = get_coordinates_google(form_addr, num, line_num, rc)

                    out_line = line.replace("\n", "") + sep + str(lon) + sep + str(lat)  + "\n"
                    out.write(out_line)
                    duplicates.add(addr)
                except:
                    errors.write(line)

            line_num += 1
    print("end num ", num)
    return rc


## Parsing using yandex api

In [14]:

def get_coordinates_yandex(addr, num, line_num, requests_count):
    """
    :param addr: адрес
    :param num: номер файла для записи
    :param line_num: номер строки
    :param requests_count: счетчик запросов
    :returns: долгота, широта, счетчик запросов
    """
    try:
        if requests_count == Y_API_MAXIMUM:
            print("limit exceeded ", num, " line_num: ", line_num)
            exit(0)
        r = requests.get(yandex_url + addr + yandex_url_end + yandex_api)
        resp=json.loads(r.text)

        form = resp["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["Point"]["pos"].split()
        lon=float(form[0])
        lat=float(form[1])
        requests_count += 1
        return lon,lat, requests_count

    except:
        print("error parsing ",num, "line num ", line_num, "addr ", addr)



In [64]:


def y_process_file(num, strt, rc, sep=';'):
    """
    :param num: номер файла для записи
    :param rc: счетчик запросов
    :param sep: разделитель csv
    :returns: счетчик запросов
    """
    print("start ", num)
    out = open(output_file, 'a')
    errors = open(errors, 'a')
    input_file, output_file = get_dirs(num)
    with open(input_file) as f:
        line_num = 0
        for line in f:
            if line_num > strt:
                if line_num == 0:
                    line_num += 1
                    out_line = line.replace("\n", "") + sep + "lon" + sep + "lat" + "\n"
                    out.write(out_line)
                    continue
                split_line = line.split(sep)
                addr = split_line[0]
                if addr not in duplicates:
                    form_addr = addr.replace("/", "%2F")
                    form_addr = form_addr.replace(" ", "+")
                    try:
                        lon, lat, rc = get_coordinates_yandex(form_addr, num, line_num, rc)

                        out_line = line.replace("\n", "") + sep + str(lon) + sep + str(lat)  + "\n"
                        out.write(out_line)
                        duplicates.add(addr)
                    except:
                        errors.write(line)

            line_num += 1
    print("end num ", num)
    return rc

Кода, вызывающего функции, нет, так как данные парсились много дней, а параметры для функций зависили от ошибок 